In [1]:
import os
os.chdir("/home/yangkai/codegen-detection/")
os.environ['CUDA_VISIBLE_DEVICES'] = "6,7"

In [2]:
from vllm import LLM
engine = LLM(model = "../llm_checkpoints/starcoder/",tensor_parallel_size=2)

/home/yangkai/anaconda3/envs/vllm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-06 15:54:47,522	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-12-06 15:54:47,891	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-12-06 15:54:50,624	INFO worker.py:1642 -- Started a local Ray instance.


INFO 12-06 15:54:51 llm_engine.py:72] Initializing an LLM engine with config: model='../llm_checkpoints/starcoder/', tokenizer='../llm_checkpoints/starcoder/', tokenizer_mode=auto, revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=2, quantization=None, seed=0)
INFO 12-06 15:55:44 llm_engine.py:205] # GPU blocks: 19088, # CPU blocks: 13107


In [6]:
import pandas as pd
data = pd.read_pickle("dataset/apps_gen_test.pkl")
data = data['extracted_full_func'].tolist()

In [7]:
import random
from vllm import SamplingParams

def mask_text(text,m=8):
    lines = text.split("\n")
    m = min(m,len(lines) // 2)

    mask_start = random.choice(list(range(len(lines) - m)))
    mask_end = mask_start + m
    prefix = "\n".join(lines[:mask_start])
    suffix = "\n".join(lines[mask_end:])

    masked_text = f"<fim_prefix>{prefix}<fim_suffix>{suffix}<fim_middle>"
    return masked_text,prefix,suffix

param = SamplingParams(
    n = 1,
    temperature=0.2,
    top_p=0.95,
    max_tokens=128
)

all_prefix = []
all_suffix = []
all_input = []
n_perturb = 16
for x in data[:10]:
    for _ in range(n_perturb):
        input,pre,suf = mask_text(x)
        all_prefix.append(pre)
        all_suffix.append(suf)
        all_input.append(input)

all_middle = engine.generate(all_input,param,use_tqdm=True)

all_perturb = []
for prefix,middle,suffix in zip(all_prefix,all_middle,all_suffix):
    try:
        assert len(middle.outputs[0].text) > 0
        perturbed_t = prefix + middle.outputs[0].text + suffix
    except:
        perturbed_t = ""

    all_perturb.append(perturbed_t)

Processed prompts: 100%|██████████| 160/160 [00:19<00:00,  8.13it/s]


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s](RayWorker pid=33583) ../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [34,0,0], thread: [96,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
(RayWorker pid=33583) ../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [34,0,0], thread: [97,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
(RayWorker pid=33583) ../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [34,0,0], thread: [98,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
(RayWorker pid=33583) ../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [34,0,0], thread: [99,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
(RayWorker pid=33583) ../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [34,0,0], thread: [100,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
(RayWorker pid=33583) ../aten/src/ATen/native/cuda/Indexing.cu:1093: inde

RayTaskError(RuntimeError): [36mray::RayWorker.execute_method()[39m (pid=33584, ip=10.82.77.54, actor_id=213b40ad79faba768064204901000000, repr=<vllm.engine.ray_utils.RayWorker object at 0x7f33242a2cd0>)
  File "/home/yangkai/anaconda3/envs/vllm/lib/python3.8/site-packages/vllm/engine/ray_utils.py", line 32, in execute_method
    return executor(*args, **kwargs)
  File "/home/yangkai/anaconda3/envs/vllm/lib/python3.8/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/home/yangkai/anaconda3/envs/vllm/lib/python3.8/site-packages/vllm/worker/worker.py", line 321, in execute_model
    output = self.model(
  File "/home/yangkai/anaconda3/envs/vllm/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/yangkai/anaconda3/envs/vllm/lib/python3.8/site-packages/vllm/model_executor/models/gpt_bigcode.py", line 252, in forward
    next_tokens = self.sa

2023-12-06 15:50:34,313	ERROR worker.py:405 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::RayWorker.execute_method() (pid=33583, ip=10.82.77.54, actor_id=f5ab32226729dcda8bcf2fcc01000000, repr=<vllm.engine.ray_utils.RayWorker object at 0x7fe448322d30>)
  File "/home/yangkai/anaconda3/envs/vllm/lib/python3.8/site-packages/vllm/engine/ray_utils.py", line 32, in execute_method
    return executor(*args, **kwargs)
  File "/home/yangkai/anaconda3/envs/vllm/lib/python3.8/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/home/yangkai/anaconda3/envs/vllm/lib/python3.8/site-packages/vllm/worker/worker.py", line 321, in execute_model
    output = self.model(
  File "/home/yangkai/anaconda3/envs/vllm/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/yangkai/anaconda3/envs/vllm/lib/python3.8/site-packages/vllm/model_execu

In [4]:
import random
def mask_text(text,m=8):
    lines = text.split("\n")
    m = min(m,len(lines) // 2)

    mask_start = random.choice(list(range(len(lines) - m)))
    mask_end = mask_start + m
    prefix = "\n".join(lines[:mask_start])
    suffix = "\n".join(lines[mask_end:])

    masked_text = f"<fim_prefix>{prefix}<fim_suffix>{suffix}<fim_middle>"
    return masked_text,prefix,suffix
mask_text("asdfa",2)

('<fim_prefix><fim_suffix>asdfa<fim_middle>', '', 'asdfa')